In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [ ]:
# Constants for the RNA sequence
FILENAME = "data/train_data_QUICK_START.csv"
VOCAB_SIZE = 5
VOCAB = {'A': 0,
         'T': 1,
         'G': 2,
         'C': 3,
         'U': 4}
SEQ_LEN = 206
EXPERIMENT_TYPE = "2A3_MaP" # DMS_MaP

# Constants for the model
TEST_SIZE = 0.2
EPOCHS = 2
BATCH_SIZE = 32

In [ ]:
# Read the CSV file and split it into train & test sets
def load_csv_data(filename=FILENAME, test_size=TEST_SIZE):
    df = pd.read_csv(filename)
    df = df.iloc[:1000, :]

    # Only load one experiment type at a time
    df = df[df['experiment_type'] == EXPERIMENT_TYPE]

    # Reformat the reactivity columns
    reactivity_columns = [col for col in df.columns if col.startswith('reactivity_0')]
    df['reactivity'] = df[reactivity_columns].values.tolist()

    # Select the relevant columns
    clean_df = df.loc[:, ['sequence', 'reactivity']]

    # Split into train and test sets
    train_df, test_df = train_test_split(clean_df, test_size=test_size)
    return train_df, test_df

train_df, test_df = load_csv_data()

In [ ]:
# One hot encode the sequence and pad it to a fixed length
def one_hot_encode(sequence):
    encoding = torch.zeros(SEQ_LEN, VOCAB_SIZE)
    for i, nucleotide in enumerate(sequence):
        encoding[i, VOCAB[nucleotide]] = 1
    return encoding

# Convert sequences and reactivities to tensors
def df_to_tensor(df):
    # Apply padded one-hot encoding to the sequences
    encoded_sequences = df['sequence'].apply(one_hot_encode)

    # Convert to tensors and standardize the shape
    sequences_tensor = torch.stack(encoded_sequences.tolist())
    reactivities_tensor = torch.tensor(df['reactivity'].values.tolist(), dtype=torch.float32)
    reactivities_tensor = reactivities_tensor.unsqueeze(-1)

    return TensorDataset(sequences_tensor, reactivities_tensor)

train_data = df_to_tensor(train_df)
test_data = df_to_tensor(test_df)

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
def test_one_hot_encode():
    sequence = "ATGCU"
    encoding = one_hot_encode(sequence)
    
    # Test the shape and make sure there's exactly 1 active column per row
    assert encoding.shape == (SEQ_LEN, VOCAB_SIZE)
    for i in range(5):
        assert encoding[i, i] == 1
        assert sum(encoding[i]) == 1

def test_df_to_tensor():
    sequences = ["UCGTA", "ATGCU"]
    reactivities = [[0.1, 0.2, 0.3, 0.4, 0.5], [0.1, 0.3, 0.5, 0.7, 0.9]]
    df = pd.DataFrame({'sequence': sequences, 'reactivity': reactivities})
    dataset = df_to_tensor(df)

    # Test the shape for both outputs
    assert len(dataset) == 2
    assert dataset[0][0].shape == (SEQ_LEN, VOCAB_SIZE)
    assert dataset[0][1].shape == (5, 1)

test_one_hot_encode()
test_df_to_tensor()

In [ ]:
# Define the Bi-LSTM model
class BiLSTM(nn.Module):
    def __init__(self, vocab_size, hidden_dim, output_dim, n_layers, bidirectional=True):
        super(BiLSTM, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.lstm = nn.LSTM(vocab_size, hidden_dim, num_layers=n_layers, bidirectional=bidirectional, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, x):
        h0 = torch.zeros(self.n_layers * 2, x.size(0), self.hidden_dim).to(x.device)  # Initial hidden state
        c0 = torch.zeros(self.n_layers * 2, x.size(0), self.hidden_dim).to(x.device)  # Initial cell state
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out)
        return out
    
model = BiLSTM(VOCAB_SIZE, hidden_dim=50, output_dim=1, n_layers=2)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Return the MAE between all yhat and y as 1 number
def loss_fn(yhat, y):
    # Clip y-values between 0 and 1
    y = torch.clamp(y, 0, 1)

    # Create a mask for non-nan values
    mask = ~torch.isnan(y)
    y[torch.isnan(y)] = 0

    # Calculate the loss at each position
    criterion = nn.L1Loss(reduction='none')
    loss = criterion(yhat, y)

    # Calculate the loss for non-nan y values
    masked_loss = torch.sum(loss * mask) / torch.sum(mask)
    return masked_loss

In [ ]:
def test_loss_fn():
    # Input same as output
    yhat = torch.tensor([[0.1], [0.2], [0.3]])
    y = torch.tensor([[0.1], [0.2], [0.3]])
    assert loss_fn(yhat, y) == 0

    # Input sometimes higher or lower
    yhat = torch.tensor([[0.1], [0.4], [0.3]])
    y = torch.tensor([[0.3], [0.2], [0.1]])
    assert loss_fn(yhat, y) == 0.2

    # y should be clamped between 0 and 1
    yhat = torch.tensor([[0], [1], [0.3]])
    y = torch.tensor([[-0.2], [1.2], [0.3]])
    assert loss_fn(yhat, y) == 0

    # The input contains multiple sequences
    yhat = torch.tensor([[[0], [1], [0.3]], [[0.1], [0.4], [0.3]]])
    y = torch.tensor([[[-0.2], [1.2], [0.3]], [[0.3], [0.2], [0.1]]])
    assert loss_fn(yhat, y) == 0.1

    # y has some nan values but no error
    yhat = torch.tensor([[0], [0.1], [0.2], [0.3]])
    y = torch.tensor([[float('nan')], [0.1], [0.2], [float('nan')]])
    assert loss_fn(yhat, y) == 0

    # y has some nan values and some error
    yhat = torch.tensor([[0.2], [0.3]])
    y = torch.tensor([[0.1], [0.2]])
    assert round(loss_fn(yhat, y).item(), 7) == 0.1

test_loss_fn()

In [ ]:
# Training loop
for epoch in range(EPOCHS):
    total_loss = 0
    for sequences, reactivities in train_loader:
        model.train()
        optimizer.zero_grad()
        outputs = model(sequences)

        print(outputs, reactivities)

        # loss = loss_fn(outputs, reactivities)
    #     total_loss += loss.item()
    #     loss.backward()
    #     optimizer.step()
    # print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")